# Web Data Scraping



## Acknowledgements

These notebooks are adaptations from a 5 session mini course at the University of Colorado. The github repo can be found [here](https://github.com/CU-ITSS/Web-Data-Scraping-S2019) [Spring 2019 ITSS Mini-Course] The course is taught by [Brian C. Keegan, Ph.D.](http://brianckeegan.com/) [Assistant Professor, Department of Information Science](https://www.colorado.edu/cmci/people/information-science/brian-c-keegan). They have been adapted for relevant content and integration with Docker so that we all have the same environment. Professor Keegan suggests using a most recent version of Python (3.7) which is set in the `requirements.txt` file.

The Spring ITSS Mini-Course was adapted from a number of sources including [Allison Morgan](https://allisonmorgan.github.io/) for the [2018 Summer Institute for Computational Social Science](https://github.com/allisonmorgan/sicss_boulder), which were in turn derived from [other resources](https://github.com/simonmunzert/web-scraping-with-r-extended-edition) developed by [Simon Munzert](http://simonmunzert.github.io/) and [Chris Bail](http://www.chrisbail.net/). 

This notebook is adapted from excellent notebooks in Dr. [Cody Buntain](http://cody.bunta.in/)'s seminar on [Social Media and Crisis Informatics](http://cody.bunta.in/teaching/2018_winter_umd_inst728e/) as well as the [PRAW documentation](https://praw.readthedocs.io/en/latest/).

## Introduction to Jupyter Notebooks

This is an example of a code cell below. You type the code into the cell and run the cell with the "Run" button in the toolbar or pressing Shift+Enter.

In [4]:
name = 'Nate Langholz'
print(name)

Nate Langholz


In [5]:
2+2

4

## Forms of structured data

There are three primary forms of structured data you will encounter on the web: HTML, XML, and JSON.

### HTML

We can use Python's `requests` library to make a valid HTTP "get" request to the Oscars' web server for the 90 Academy Awards which will return the raw HTML. There are more than 144,000 characters in the document!

In [6]:
import requests

# Pretend to be a web browser and make a get request of a webpage
oscars90_request = requests.get('https://www.oscars.org/oscars/ceremonies/2018')

# The .text returns the text from the request
oscars90_html = oscars90_request.text

# The oscars90_html is a string, we can use the common len function to ask how long the string is (in characters)
len(oscars90_html)

159082

Let's look at the first thousand characters. Mostly declarations to handle Internet Explorer's notorious refusal to follow standards—stuff you don't need to worry about.

In [7]:
# The [0:1000] is a slicing notation 
# It gets the first (position 0 in Python) character until the 1000th character

print(oscars90_html[0:1000])

<!DOCTYPE html>
  <!--[if IEMobile 7]><html class="no-js ie iem7" lang="en" dir="ltr"><![endif]-->
  <!--[if lte IE 6]><html class="no-js ie lt-ie9 lt-ie8 lt-ie7" lang="en" dir="ltr"><![endif]-->
  <!--[if (IE 7)&(!IEMobile)]><html class="no-js ie lt-ie9 lt-ie8" lang="en" dir="ltr"><![endif]-->
  <!--[if IE 8]><html class="no-js ie lt-ie9" lang="en" dir="ltr"><![endif]-->
  <!--[if (gte IE 9)|(gt IEMobile 7)]><html class="no-js ie" lang="en" dir="ltr" prefix="og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#"><![endif]-->
  <!--[if !IE]><!--><html class="no-js" lang="en" dir="ltr" prefix="og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#"><!--<![endif]-->
<head>
  <meta charset="utf-8" /><script type="text/javascript

Looking at 1,000 lines about a third of the way through the document, we can see some of the structure we found with the "Inspect" tool above corresponding to the closing lines of the "Actor in a Supporting Role" grouping and the opening lines of the "Acress in a Leading Role" grouping.

In [8]:
# You can slice any ranges you'd like up, as long as it's not beyond the length of the string
# oscars90_html[144588:] would return an error

print(oscars90_html[50000:51000])

esc field--type-text-long field--label-hidden ellipsis"><div class="field-items"><div class="field-item even">Lee Unkrich and Darla K. Anderson</div></div></div>  </div>
</div>
</div></li>
</ul></div>  </div>  
  <div class="views-field views-field-field-more-highlights">        <div class="field-content"><a href="https://www.oscars.org/oscars/ceremonies/90th-oscar-highlights" class="btn-link">View More Highlights</a></div>  </div>  
  <div class="views-field views-field-field-memorable-moments">    <span class="views-label views-label-field-memorable-moments">Memorable Moments</span>    <div class="field-content"><ul><li><div class="field-collection-view clearfix view-mode-full"><div class="entity entity-field-collection-item field-collection-item-field-memorable-moments clearfix" class="entity entity-field-collection-item field-collection-item-field-memorable-moments">
  <div class="content">
    <div class="field field--name-field-ceremonies-media field--type-image field--label-hidd

We're not actually going to be slicing the text to get this structured data out, we'll use a wonderful tool call [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) to do the heavy lifting for us.

### XML

XML has taken on something of an afterlife as the official data standard for the U.S. Congress. The [House](http://clerk.house.gov/index.aspx) and [Senate](https://www.senate.gov/general/XML.htm) both release information about members, committees, schedules, legislation, and votes in XML. These are immaculately formatted and documented and remarkably up-to-date: the data for members of the 116th Congress are already posted.

Use the `requests` library to make a HTTP get request to the House's webserver and get the list of current member data.

In [9]:
house_raw = requests.get('http://clerk.house.gov/xml/lists/MemberData.xml').text

senate_raw = requests.get('https://www.senate.gov/legislative/LIS_MEMBER/cvc_member_data.xml').text

This data is still in a string format (`type(house_raw)`), so it's difficult to search and navigate. Let's make our first soup together using [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

In [10]:
# First import the library
from bs4 import BeautifulSoup

# Then make the soup, specifying the "lxml" parser
house_soup = BeautifulSoup(house_raw,'lxml')

What's so great about this soup-ified string? We now have a suite of new functions and methods that let us navigate the tree. First, let's inspect the different tags/elements in this tree of House member data. This is the full tree of data.

In [11]:
# Make an empty list to store data
children = []

# Start a loop to go through all the children tags in house_soup
for tag in house_soup.findChildren():
    
    # If the name of the tag (tag.name) is not already in the children list
    if tag.name not in children:
        
        # Add the name of the tag to the children list
        children.append(tag.name)

# Look at the list members
children

['html',
 'body',
 'memberdata',
 'title-info',
 'congress-num',
 'congress-text',
 'session',
 'majority',
 'minority',
 'clerk',
 'weburl',
 'members',
 'member',
 'statedistrict',
 'member-info',
 'namelist',
 'bioguideid',
 'lastname',
 'firstname',
 'middlename',
 'sort-name',
 'suffix',
 'courtesy',
 'prior-congress',
 'official-name',
 'formal-name',
 'party',
 'caucus',
 'state',
 'state-fullname',
 'district',
 'townname',
 'office-building',
 'office-room',
 'office-zip',
 'office-zip-suffix',
 'phone',
 'elected-date',
 'sworn-date',
 'committee-assignments',
 'committee',
 'subcommittee',
 'footnote-ref',
 'footnote',
 'predecessor-info',
 'pred-lastname',
 'pred-firstname',
 'pred-middlename',
 'pred-official-name',
 'pred-formal-name',
 'pred-title',
 'pred-memindex',
 'pred-sort-name',
 'pred-party',
 'pred-vacate-date',
 'pred-footnote-ref',
 'pred-footnote',
 'committees',
 'committee-fullname',
 'ratio',
 'subcommittee-fullname']

We can navigate through the tree. You won't do this in practice, but it's helpful for debugging. In this case, we navigated from the root node (`html`) into the `body` tag, then the `memberdata` tag, then the `members` tag. There are 441 descendents at this level, corresponding to the 435 voting seats and the 6 seats for territories.

In [12]:
len(house_soup.html.body.memberdata.members)

441

You can also short-cut to the members tag directly rather than navigating down the parent elements.

In [13]:
len(house_soup.members)

441

The `.contents` method is great for getting a list of the children below the tag as a list. We can use the `[0]` slice to get the first member and their data in the list. Interestingly, the `<committee-assignments>` tags are currently empty since these have not yet been allocated, but will in the next few weeks.

In [14]:
house_soup.members.contents[0]

<member><statedistrict>AK00</statedistrict><member-info><namelist>Young, Don</namelist><bioguideid>Y000033</bioguideid><lastname>Young</lastname><firstname>Don</firstname><middlename></middlename><sort-name>YOUNG,DON</sort-name><suffix></suffix><courtesy>Mr.</courtesy><prior-congress>115</prior-congress><official-name>Don Young</official-name><formal-name>Mr. Young</formal-name><party>R</party><caucus>R</caucus><state postal-code="AK"><state-fullname>Alaska</state-fullname></state><district>At Large</district><townname>Fort Yukon</townname><office-building>RHOB</office-building><office-room>2314</office-room><office-zip>20515</office-zip><office-zip-suffix>0200</office-zip-suffix><phone>(202) 225-5765</phone><elected-date date="20181106">November  6, 2018</elected-date><sworn-date date="20190103">January  3, 2019</sworn-date></member-info><committee-assignments><committee comcode="II00" rank="2"></committee><committee comcode="PW00" rank="2"></committee><subcommittee rank="1" subcomcod

You could keep navigating down the tree from here.

In [15]:
house_soup.members.contents[0].bioguideid

<bioguideid>Y000033</bioguideid>

Note that this navigation method breaks when the tag has a hyphen in it.

In [16]:
house_soup.members.contents[0].state-fullname

NameError: name 'fullname' is not defined

Instead you can use the `.find()` method to handle these hyphenated cases.

In [17]:
house_soup.members.contents[0].find('state-fullname')

<state-fullname>Alaska</state-fullname>

We can access the text inside the tag with `.text`

In [18]:
house_soup.members.contents[0].find('state-fullname').text

'Alaska'

The `.find_all()` method will be your primary tool when working with structured data. The `<party>` tag codes party membership (D=Democratic, R=Republican) for each representative. 

In [19]:
house_soup.find_all('party')[:10]

[<party>R</party>,
 <party>R</party>,
 <party>R</party>,
 <party>R</party>,
 <party>R</party>,
 <party>R</party>,
 <party>R</party>,
 <party>D</party>,
 <party>R</party>,
 <party>R</party>]

There [should be](https://en.wikipedia.org/wiki/116th_United_States_Congress#Party_summary) 235 Democrats and 199 Republicans, plus the other non-voting members from territories.

In [20]:
# Initialize a counter
democrats = 0
republicans = 0
other = 0

# Loop through each element of the caucus tags
for p in house_soup.find_all('party'):
    
    # Check if it's D, R, or something else
    if p.text == "D":
        
        # Increment the appropriate counter
        democrats += 1
    
    elif p.text == "R":
        republicans += 1
    else:
        other += 1
        
print("There are {0} Democrats, {1} Republicans, and {2} others in the 116th Congress.".format(democrats,republicans,other))


There are 239 Democrats, 199 Republicans, and 3 others in the 116th Congress.


### JSON

JSON is attractive for programmers using JavaScript and Python because it can represent a mix of different data types. We need to make a brief digression into Python's fundamental data stuctures in order to understand the contemporary attraction to JSON. Python has a few fundamental data types for representing collections of information:

* **Lists**: This is a basic ordered data structure that can contain strings, ints, and floats.
* **Dictionaries**: This is an unordered data structure containing key-value pairs, like a phonebook.

Let's look at some examples of lists and dictionaries and then we can try the exercises below.

### Exercises

Below is an example of a [tweet status](https://dev.twitter.com/overview/api/tweets) object that Twitter's [API returns](https://dev.twitter.com/rest/reference/get/statuses/show/id). This `obama_tweet` dictionary corresponds to [this tweet](https://twitter.com/BarackObama/status/831527113211645959). This is a classic example of a JSON object containing a mixture of dictionaries, lists, lists of dictionaries, dictionaries of lists, *etc*.

In [22]:
obama_tweet = {'created_at': 'Tue Feb 14 15:34:47 +0000 2017',
               'favorite_count': 1023379,
               'hashtags': [],
               'id': 831527113211645959,
               'id_str': '831527113211645959',
               'lang': 'en',
               'media': [{'display_url': 'pic.twitter.com/O0UhJWoqGN',
                          'expanded_url': 'https://twitter.com/BarackObama/status/831527113211645959/photo/1',
                          'id': 831526916398149634,
                          'media_url': 'http://pbs.twimg.com/media/C4otUykWcAIbSy1.jpg',
                          'media_url_https': 'https://pbs.twimg.com/media/C4otUykWcAIbSy1.jpg',
                          'sizes': {'large': {'h': 800, 'resize': 'fit', 'w': 1200},
                                    'medium': {'h': 800, 'resize': 'fit', 'w': 1200},
                                    'small': {'h': 453, 'resize': 'fit', 'w': 680},
                                    'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
                          'type': 'photo',
                          'url': 'https://t.co/O0UhJWoqGN'}],
               'retweet_count': 252266,
               'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
               'text': 'Happy Valentine’s Day, @michelleobama! Almost 28 years with you, but it always feels new. https://t.co/O0UhJWoqGN',
               'urls': [],
               'user': {'created_at': 'Mon Mar 05 22:08:25 +0000 2007',
                        'description': 'Dad, husband, President, citizen.',
                        'favourites_count': 10,
                        'followers_count': 84814791,
                        'following': True,
                        'friends_count': 631357,
                        'id': 813286,
                        'lang': 'en',
                        'listed_count': 221906,
                        'location': 'Washington, DC',
                        'name': 'Barack Obama',
                        'profile_background_color': '77B0DC',
                        'profile_background_image_url': 'http://pbs.twimg.com/profile_background_images/451819093436268544/kLbRvwBg.png',
                        'profile_banner_url': 'https://pbs.twimg.com/profile_banners/813286/1484945688',
                        'profile_image_url': 'http://pbs.twimg.com/profile_images/822547732376207360/5g0FC8XX_normal.jpg',
                        'profile_link_color': '2574AD',
                        'profile_sidebar_fill_color': 'C2E0F6',
                        'profile_text_color': '333333',
                        'screen_name': 'BarackObama',
                        'statuses_count': 15436,
                        'time_zone': 'Eastern Time (US & Canada)',
                        'url': 'https://t.co/93Y27HEnnX',
                        'utc_offset': -18000,
                        'verified': True},
               'user_mentions': [{'id': 409486555,
                                  'name': 'Michelle Obama',
                                  'screen_name': 'MichelleObama'}]}

1. What are the top-most keys in the `obama_tweet` object?
2. When was this tweet sent?
3. Does this tweet mention anyone?
4. How many retweets did this tweet receive (at the time I collected it)?
5. How many followers does the "user" who wrote this tweet have?
6. What's the "media_url" for the image in this tweet?

In [24]:
obama_tweet.keys()

dict_keys(['created_at', 'favorite_count', 'hashtags', 'id', 'id_str', 'lang', 'media', 'retweet_count', 'source', 'text', 'urls', 'user', 'user_mentions'])

In [26]:
obama_tweet.values()

dict_values(['Tue Feb 14 15:34:47 +0000 2017', 1023379, [], 831527113211645959, '831527113211645959', 'en', [{'display_url': 'pic.twitter.com/O0UhJWoqGN', 'expanded_url': 'https://twitter.com/BarackObama/status/831527113211645959/photo/1', 'id': 831526916398149634, 'media_url': 'http://pbs.twimg.com/media/C4otUykWcAIbSy1.jpg', 'media_url_https': 'https://pbs.twimg.com/media/C4otUykWcAIbSy1.jpg', 'sizes': {'large': {'h': 800, 'resize': 'fit', 'w': 1200}, 'medium': {'h': 800, 'resize': 'fit', 'w': 1200}, 'small': {'h': 453, 'resize': 'fit', 'w': 680}, 'thumb': {'h': 150, 'resize': 'crop', 'w': 150}}, 'type': 'photo', 'url': 'https://t.co/O0UhJWoqGN'}], 252266, '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'Happy Valentine’s Day, @michelleobama! Almost 28 years with you, but it always feels new. https://t.co/O0UhJWoqGN', [], {'created_at': 'Mon Mar 05 22:08:25 +0000 2007', 'description': 'Dad, husband, President, citizen.', 'favourites_count': 10, 'followers_count'

In [27]:
obama_tweet['created_at']

'Tue Feb 14 15:34:47 +0000 2017'

In [28]:
obama_tweet['user_mentions'] # access the dictory 

[{'id': 409486555, 'name': 'Michelle Obama', 'screen_name': 'MichelleObama'}]

In [29]:
obama_tweet.get('user_mentions') # access the object # in python it does not really matter 

[{'id': 409486555, 'name': 'Michelle Obama', 'screen_name': 'MichelleObama'}]